# 割当問題

In [1]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from ortools.math_opt.python import mathopt

In [2]:
n = 1000
np.random.seed(seed=0)
cost = np.random.randint(100, 1000, size=(n, n))
v = list(range(n))

In [3]:
%%time
row_ind, col_ind = linear_sum_assignment(cost)
print(cost[row_ind, col_ind].sum())

100961
CPU times: user 18.4 ms, sys: 704 μs, total: 19.1 ms
Wall time: 18.6 ms


In [4]:
model = mathopt.Model(name="assign")

x = {}
for i in v:
    for j in v:
        x[i, j] = model.add_binary_variable(name=f"x[{i},{j}]")

for j in v:
    model.add_linear_constraint(sum(x[i, j] for i in v) == 1)
for i in v:
    model.add_linear_constraint(sum(x[i, j] for j in v) == 1)

model.minimize(sum(cost[i, j] * x[i, j] for i in v for j in v))

In [5]:
%%time
params = mathopt.SolveParameters(enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.GSCIP, params=params)
if result.termination.reason != mathopt.TerminationReason.OPTIMAL:
    raise RuntimeError(f"model failed to solve: {result.termination}")

presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 2000 upgd conss, 0 impls, 2000 clqs
   (5.5s) probing: 51/1000000 (0.0%) - 0 fixings, 0 aggregations, 0 implications, 0 bound changes
   (5.5s) probing aborted: 50/50 successive totally useless probings
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
   Deactivated symmetry handling methods, since SCIP was built without symmetry detector (SYM=none).
presolving (2 rounds: 2 fast, 2 medium, 2 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 2000 cliques
presolved problem has 1000000 variables (1000000 bin, 0 int, 0 impl, 0 cont) and 2000 constraints
   2000 constraints of type <setppc>
transformed objective value is always integral (scale: 1)
Presolving Time: 5.44

 time | node  | left  |LP iter|LP it/n|mem/he

In [6]:
%%time
params = mathopt.SolveParameters(enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.HIGHS, params=params)
if result.termination.reason != mathopt.TerminationReason.OPTIMAL:
    raise RuntimeError(f"model failed to solve: {result.termination}")

Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+02, 1e+03]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
2000 rows, 1000000 cols, 2000000 nonzeros  0s
2000 rows, 1000000 cols, 2000000 nonzeros  21s
Objective function is integral with scale 1

Solving MIP model with:
   2000 rows
   1000000 cols (1000000 binary, 0 integer, 0 implied int., 0 continuous)
   2000000 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0    23.7s
 T       0       0         0   0.00%   0               100961           100.00%        0      0      0      9204    28.6s

Solving report
  Status            Optimal
  Primal bound      100961
  Dual bound        100961
  Gap     

In [7]:
%%time
params = mathopt.SolveParameters(enable_output=True)
result = mathopt.solve(model, mathopt.SolverType.CP_SAT, params=params)
if result.termination.reason != mathopt.TerminationReason.OPTIMAL:
    raise RuntimeError(f"model failed to solve: {result.termination}")


Running basic LP presolve, initial problem dimensions: 2000 rows, 1000000 columns, 2000000 entries with magnitude in [1.000000e+00, 1.000000e+00]
glop::FixedVariablePreprocessor                        2000 rows, 1000000 columns, 2000000 entries with magnitude in [1.000000e+00, 1.000000e+00]
glop::SingletonPreprocessor                            2000 rows, 1000000 columns, 2000000 entries with magnitude in [1.000000e+00, 1.000000e+00]
glop::ForcingAndImpliedFreeConstraintPreprocessor      2000 rows, 1000000 columns, 2000000 entries with magnitude in [1.000000e+00, 1.000000e+00]
glop::FreeConstraintPreprocessor                       2000 rows, 1000000 columns, 2000000 entries with magnitude in [1.000000e+00, 1.000000e+00]
glop::UnconstrainedVariablePreprocessor                2000 rows, 1000000 columns, 2000000 entries with magnitude in [1.000000e+00, 1.000000e+00]

Scaling to pure integer problem.
Num integers: 1000000/1000000 (implied: 0 in_inequalities: 0 max_scaling: 0) [IP] 
Maximu